In [1]:
import os
import sys
sys.path.insert(1, '../../../..')

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.FNO import FNO1d
from methods.methodsDataset.FNODataset import FNODataset
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [3]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1e-3, 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_FNO = {'layer_dims': None, 'FourierLayers_modes': None, 'device': device, 'seed': 123}

In [4]:
def delta(y, x, dy = 1., dx = 0.) :
    """    
    y : int, float or ndarray of size 1
    x : ndarray
    
    return dy if x = y and dx otherwise
    """
    if torch.is_tensor(y) or torch.is_tensor(y):
        return torch.where(x == y, dy, dx)
    return np.where(x == y, dy, dx)

In [5]:
params_solver['source_term'] = lambda y, x : delta(y, x)
params_solver['y'] = None

In [6]:
solver = Solver(params={'solver': params_solver})

In [7]:
Y_list = solver.x[20:81].reshape(-1, 1)

In [8]:
U_sols = []
for y in Y_list:
    solver.change_y(new_y= y)
    U_sols.append(solver.solve())
U_sols = np.stack(U_sols)

In [9]:
Y_train, Y_val, U_train, U_val = sklearn.model_selection.train_test_split(Y_list, U_sols, test_size=0.2, random_state=123)

In [10]:
delta_Y_train = delta(Y_train, solver.x)
delta_Y_val = delta(Y_val, solver.x)

In [11]:
Y_fno_train = torch.Tensor(delta_Y_train).unsqueeze(-1)
Y_fno_val = torch.Tensor(delta_Y_val).unsqueeze(-1)

In [12]:
def get_dataset():
    return Y_fno_train, Y_fno_val, U_train, U_val

In [13]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [14]:
def define_model(trial, input_chanel):
    n_fourier_layers = trial.suggest_int("n_fourier_layers", 1, 5)
    lifting_layer = trial.suggest_int("lifting_layer", 32, 64)
    layers = [input_chanel, lifting_layer]
    fourier_modes = []
    for i in range(n_fourier_layers):
        out_features = trial.suggest_int("Fourier_units_l{}".format(i), 32, 64)
        fourier_mode = trial.suggest_int("Fourier_modes_l{}".format(i), 12, 16)
        layers += [out_features]
        fourier_modes.append(fourier_mode)
    
    params_FNO_trial = copy.deepcopy(params_FNO)
    params_FNO_trial['layers_dim'] = layers
    params_FNO_trial['FourierLayers_modes'] = fourier_modes
    return FNO1d(params={'solver':params_solver, 'method':params_FNO_trial})

In [15]:
def objective(trial):
    # Generate the model.
    model = define_model(trial, 1)

    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)

    Y_train, Y_val, U_train, U_val = get_dataset()
    
    batch_size = trial.suggest_int("batch_size", 128, 1024)
    
    n_epochs = 10000
    hyperparameters = {'lr': lr, 'epochs': n_epochs, 'optimizer': 'Adam', 'batch_size': batch_size}
    
    torch.manual_seed(params_FNO['seed'])
    model.fit(hyperparameters=hyperparameters, 
                            D_train=Y_train, D_val=Y_val, U_train=U_train, U_val=U_val)

            
    val_loss = model.loss_dict['val']
    trial.report(min(val_loss), n_epochs)
    
    return min(val_loss)

In [16]:
sampler = TPESampler(seed=params_FNO['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-10 17:42:47,121] A new study created in memory with name: no-name-d5df1679-1c4b-4819-b68c-166ef61c3736


In [17]:
study.optimize(objective, n_trials=5)

[tr : 1.1e-06, val : 1.4e-06]: 100%|█████| 10000/10000 [00:52<00:00, 190.00it/s]
[I 2023-08-10 17:43:40,500] Trial 0 finished with value: 1.264178536075633e-06 and parameters: {'n_fourier_layers': 4, 'lifting_layer': 41, 'Fourier_units_l0': 39, 'Fourier_modes_l0': 14, 'Fourier_units_l1': 55, 'Fourier_modes_l1': 14, 'Fourier_units_l2': 64, 'Fourier_modes_l2': 15, 'Fourier_units_l3': 47, 'Fourier_modes_l3': 13, 'lr': 0.00010703702960745551, 'batch_size': 781}. Best is trial 0 with value: 1.264178536075633e-06.
[tr : 6.5e-07, val : 8.2e-07]: 100%|█████| 10000/10000 [00:43<00:00, 228.10it/s]
[I 2023-08-10 17:44:24,347] Trial 1 finished with value: 7.380872375506442e-07 and parameters: {'n_fourier_layers': 3, 'lifting_layer': 33, 'Fourier_units_l0': 45, 'Fourier_modes_l0': 15, 'Fourier_units_l1': 38, 'Fourier_modes_l1': 12, 'Fourier_units_l2': 49, 'Fourier_modes_l2': 14, 'lr': 0.0008002079756292469, 'batch_size': 889}. Best is trial 1 with value: 7.380872375506442e-07.
[tr : 5.1e-07, val : 

In [18]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  5
Best trial:
  Value:  4.832523359254992e-07
  Params: 
    n_fourier_layers: 5
    lifting_layer: 40
    Fourier_units_l0: 47
    Fourier_modes_l0: 16
    Fourier_units_l1: 49
    Fourier_modes_l1: 15
    Fourier_units_l2: 35
    Fourier_modes_l2: 16
    Fourier_units_l3: 51
    Fourier_modes_l3: 14
    Fourier_units_l4: 43
    Fourier_modes_l4: 13
    lr: 0.0001782652255813462
    batch_size: 739


In [19]:
modes = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'Fourier':
        if key.split('_')[1] == 'modes':
            modes.append(value)
        if key.split('_')[1] == 'units':
            layer_dims.append(value)
    elif key.split('_')[0] == 'lifting':
        lifting = value
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value
    elif key == 'batch_size':
        batch_size = value

Use this to fill config_step_1.py

In [20]:
print(layer_dims, '\n', modes, '\n', lifting, '\n', '\n', lr, '\n', batch_size)

[47, 49, 35, 51, 43] 
 [16, 15, 16, 14, 13] 
 40 
 
 0.0001782652255813462 
 739
